In [1]:

import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile as tif

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour

from caiman.source_extraction import cnmf as cnmf
from scipy.sparse import csc_matrix

import h5py
bpl.output_notebook()

C:\Anaconda3\envs\py36\lib\site-packages\scipy\ndimage\morphology.py:114: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[slc] = structure != 0


Loading BokehJS ...

In [2]:
cnm = cnmf.online_cnmf.load_OnlineCNMF('C:/Users/bnste/Documents/scripts/jon_2p_data/analysis_002.hdf5')

C:\Anaconda3\envs\py36\lib\site-packages\scipy\ndimage\morphology.py:114: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[slc] = structure != 0


In [3]:
cnm.estimates.nb_view_components()

In [4]:
cnm.estimates.play_movie()

TypeError: play_movie() missing 1 required positional argument: 'imgs'

In [5]:

def cellInfoCaimanHdf5(hdf5File):
    """
    Given a CaImAn output .hdf5 file, returns the spatial cell profiles, cell signal traces, and background profiles and traces
    Output:
    (cellProfs, signalTraces, bgProfs, bgTraces)
    cellProfs is a numCells x xDim x yDim array of spatial cell profiles
    signalTraces is a  tBins x numCells array of fluorescence traces from each cell
    bgProfs: nBGComps x xDim x yDim
    bgTraces: tBins x nBGComps
    """
    cellProfs = []
    signalTraces = []

    with h5py.File(hdf5File, mode='r') as cFile:
        est = cFile['estimates']
        Ainfo = est['A']
        signalTraces = np.transpose( np.array(est['C']) )
        Adata = np.array(Ainfo['data'])
        Aindices = np.array(Ainfo['indices'])
        Aindptr = np.array(Ainfo['indptr'])
        Ashape = np.array(Ainfo['shape'])
        A = csc_matrix((Adata,Aindices,Aindptr) , shape=Ashape ).transpose()
        A = np.array(A.todense())
        cellProfs = A.reshape((A.shape[0],np.array(est['dims'])[0] ,-1))
        f = np.array(est['f']).transpose()
        b = np.array(est['b']).transpose()
        b = b.reshape(b.shape[0],cellProfs.shape[1],cellProfs.shape[2])
    return (cellProfs, signalTraces, b, f)

A, C, b, f = cellInfoCaimanHdf5('C:/Users/bnste/Documents/scripts/jon_2p_data/analysis_002.hdf5')
#A, C, b, f = cellInfoCaimanHdf5(r'C:\Users\bnste\Documents\segmentation_project\neurofinder\data\neurofinder.00.02\results\results.hdf5')

(336, 108)

In [8]:
tmax = 600
mov =  ( np.tensordot(C[:tmax], A, (-1,0) ) + np.tensordot(f[:tmax],b,(-1,0))).transpose((0,2,1))
#mov = .25 * (mov[:-3] + mov[1:-2] + mov[2:-1] + mov[3:])
mov_cm = cm.movie(mov, fr=30)
mov_cm.play(magnification=1, gain=2)

In [18]:
tif.imsave('C:/Users/bnste/Documents/scripts/sample_caiman_output_movie.tif', np.expand_dims(mov.astype('float32'),1), imagej=True)